# Trabalho prático de Análise Inteligente em Sistemas de "Big Data"

Importar bibliotecas

In [19]:
import pandas as pd
import time
from pymongo import MongoClient
from pyspark.sql import SparkSession

Juntar os datasets e medir o tempo de execução com Python

In [20]:
# Measure exec time: Start point
start_time = time.time()

# Read the data from the CSV files
ds1 = pd.read_excel('dataset_input/efeito_estufa.xlsx')
ds2 = pd.read_excel('dataset_input/emissoes_gases.xlsx')
ds3 = pd.read_excel('dataset_input/intensidade_carbonica.xlsx')
ds4 = pd.read_excel('dataset_input/PIBcrescimento.xlsx')
ds5 = pd.read_excel('dataset_input/PIBdespesas.xlsx')
ds6 = pd.read_excel('dataset_input/PIBproducao.xlsx')
ds7 = pd.read_excel('dataset_input/PIBrendimento.xlsx')

# Merge the data
dsAmbiente = pd.merge(pd.merge(ds1, ds2, on='Anos'), ds3, on='Anos')
dsPIB = pd.merge(pd.merge(ds4, ds5, on='Anos'), pd.merge(ds6, ds7, on='Anos'), on='Anos')

# verify joined datasets
dsPIB.dropna(inplace=True)
# drop repeated columns total, total_x and total_y are all the same values
dsPIB.drop(columns=['Total_x', 'Total_y'], inplace=True)

dsAmbiente.dropna(inplace=True)

ds = pd.merge(dsAmbiente, dsPIB, on='Anos')

ds.to_csv('dataset_output/output.csv', index=False)

# Measure exec time: End point
end_time = time.time()

# Calculate execution time and show in stdout
exec_time = end_time - start_time
print(f"Execution time: {round(exec_time, 3)} seconds")

Execution time: 0.257 seconds


Utilizar o PySpark para medição do tempo de execução, com conversão do dataset para formato Parquet (*Nota: Aparente erro na função write*)

In [21]:
session = SparkSession.builder.appName("CSV to Parquet").getOrCreate()

df_spark = session.read.csv("dataset_output/output.csv", header=True, inferSchema=True)

df_spark.write.parquet("dataset_output/output.parquet") # Write pode não estar funcional

session.stop()

Py4JJavaError: An error occurred while calling o157.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(Fi

Introdução do ficheiro ~~Parquet~~ (*not working*) CSV para uma base de dados (MongoDB)

In [22]:
# Connect to MongoDB

from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["big_data"]
collection = db["assignment"]

df = pd.read_csv("dataset_output/output.csv")
dictionary = df.to_dict(orient="records")

collection.insert_many(dictionary)

InsertManyResult([ObjectId('65f33952d6202e899881b445'), ObjectId('65f33952d6202e899881b446'), ObjectId('65f33952d6202e899881b447'), ObjectId('65f33952d6202e899881b448'), ObjectId('65f33952d6202e899881b449'), ObjectId('65f33952d6202e899881b44a'), ObjectId('65f33952d6202e899881b44b'), ObjectId('65f33952d6202e899881b44c'), ObjectId('65f33952d6202e899881b44d'), ObjectId('65f33952d6202e899881b44e'), ObjectId('65f33952d6202e899881b44f'), ObjectId('65f33952d6202e899881b450'), ObjectId('65f33952d6202e899881b451'), ObjectId('65f33952d6202e899881b452'), ObjectId('65f33952d6202e899881b453'), ObjectId('65f33952d6202e899881b454'), ObjectId('65f33952d6202e899881b455'), ObjectId('65f33952d6202e899881b456'), ObjectId('65f33952d6202e899881b457'), ObjectId('65f33952d6202e899881b458'), ObjectId('65f33952d6202e899881b459'), ObjectId('65f33952d6202e899881b45a'), ObjectId('65f33952d6202e899881b45b'), ObjectId('65f33952d6202e899881b45c'), ObjectId('65f33952d6202e899881b45d'), ObjectId('65f33952d6202e899881b4